In [1]:
import json
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from numpy import dot
from numpy.linalg import norm
from operator import itemgetter
import requests
import folium
import pandas as pd
import random

In [2]:
#코사인 유사도 구하기
def cosine_similarity(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [128]:
#임의로 선택
user = "액티비티, 레저/체험, 아이"

#star_type   #아침시간인지 점심 시간인지 T/F
start_time  = 8  #시작시간 받아 오기
lunch_time = 12
morining_time = 0
total_time = 0

if start_time <= 9 :
   morning_time = (lunch_time - start_time) * 3600
   start_type = True
else :
   start_type = False


#유사도를 추가해 사용할 리스트
place = []

#장소들의 태그들을 모두 저장할 리스트
tag = []
with open('data_ver4.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
    #모든 장소의 태그들 추가
    for i in range(len(data["items"])) :
        tag.append(data["items"][i]["tag"])
    
    #마지막에 유저 태그 추가
    tag.append(user)

    #태그 벡터화(코사인 유사도를 구하기 위해)
    tfidf_vect_place = CountVectorizer()
    place_vector = tfidf_vect_place.fit_transform(tag)

    place_vector_dense = place_vector.todense()

    #장소별 태그와 유저 태그 비교 (코사인 유사도 이용)
    for j in range(len(data["items"])):
        place_vector = np.array(place_vector_dense[j]).reshape(-1,)
        user_vec = np.array(place_vector_dense[len(data["items"])-1]).reshape(-1,)

        similarity_tag = cosine_similarity(place_vector, user_vec)

        #1에 가까울수록 유사함

        #장소별 유사도 추가해 저장
        place.append(data["items"][j])
        place[j]["distance"] = similarity_tag



    #유사도 높은 순으로 정렬
    sort_place = sorted(place, key=itemgetter('distance'), reverse=True) 

    #사용자의 태그와 장소 태그의 유사도(상위 50개)
    for k in range(50):
        print('장소: {0} '.format(sort_place[k]["title"]) +' Cosine 유사도: {0:.3f}'.format(sort_place[k]["distance"]))

장소: 우도잠수함  Cosine 유사도: 1.000
장소: 퍼시픽리솜 요트투어샹그릴라  Cosine 유사도: 0.920
장소: 제주빅볼랜드  Cosine 유사도: 0.818
장소: 돈내코유원지  Cosine 유사도: 0.806
장소: 돈내코유원지  Cosine 유사도: 0.804
장소: 렛츠런파크제주  Cosine 유사도: 0.783
장소: 조랑말체험공원  Cosine 유사도: 0.783
장소: 세리월드 테마파크  Cosine 유사도: 0.783
장소: 타보카 투명카약  Cosine 유사도: 0.763
장소: 월정투명카약  Cosine 유사도: 0.763
장소: 김녕요트투어  Cosine 유사도: 0.763
장소: 성읍랜드  Cosine 유사도: 0.753
장소: 노루생태관찰원  Cosine 유사도: 0.753
장소: 제주레포츠랜드  Cosine 유사도: 0.725
장소: 더마파크  Cosine 유사도: 0.701
장소: 곽지해수욕장  Cosine 유사도: 0.678
장소: 금능해수욕장  Cosine 유사도: 0.678
장소: 비체올린  Cosine 유사도: 0.658
장소: 제주카약올레  Cosine 유사도: 0.645
장소: 다이나믹메이즈 제주도성읍점  Cosine 유사도: 0.645
장소: 제주라프  Cosine 유사도: 0.609
장소: 안덕계곡  Cosine 유사도: 0.609
장소: 김녕해수욕장  Cosine 유사도: 0.607
장소: 쇠소깍  Cosine 유사도: 0.603
장소: 제주실탄사격장  Cosine 유사도: 0.585
장소: 9.81파크  Cosine 유사도: 0.585
장소: 황우지해안  Cosine 유사도: 0.585
장소: 협재해수욕장  Cosine 유사도: 0.579
장소: 제주불빛정원  Cosine 유사도: 0.572
장소: 귤밭76번지  Cosine 유사도: 0.570
장소: 함덕해수욕장  Cosine 유사도: 0.566
장소: 신양섭지해수욕장  Cosine 유사도: 0.564
장소: 하도해변  Cosine 유사도: 0.564

In [129]:
#두 지점의 거리 계산(km)
def cal_dist(start_latitude,start_longitude, end_latitude,end_longitude):
    return (((end_latitude - start_latitude)*88.8)**2 +((end_longitude - start_longitude)*88.8)**2)**(1/2)

In [130]:
#시작: 공항 좌표
start_latitude = 33.5059364682672
start_longitude = 126.495951277797

print(type(sort_place[0]["latitude"]))
min = 10000

#선택된 장소를 저장할 딕셔너리
end_place = {}
last_end_place = {}

#move_time =      이동시간을 저장


#api를 통해 얻어온 정보를 저장할 리스트
doc = []
num = None


#긱 징소 중 시작 장소와 제일 가까운 곳을 선택
for k in range(50):
    #거리 계산
    dist = cal_dist(start_latitude,start_longitude, sort_place[k]["latitude"],sort_place[k]["longitude"])
    if dist >= 1 and dist <= 10:
        if dist <= min:
            min = dist
            end_place = sort_place[k]
            num = k

#장소 중복을 피하기 위해 
sort2_place = sort_place[:50]
del sort2_place[num]


#선택된 장소 저장
last_end_place = end_place


# REST 키
rest_api_key = 'b160df784ddfc397d6fe91d51bc8d051'

headers = {"Authorization" : "KakaoAK {}".format(rest_api_key)}

url = "https://apis-navi.kakaomobility.com/v1/directions?origin={0},{1}&destination={2},{3}".format(start_longitude,start_latitude,end_place["longitude"], end_place["latitude"]) + "&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"

# GET을 이용하여 정보 불러오기
res = requests.get(url, headers=headers)

# Json 형식으로 불러오기
#불러온 이동정보를 doc에 추가
doc.append(json.loads(res.text))

#깂을 제대로 불러왔는지 확인
print(doc[0]["routes"][0]["result_code"], doc[0]["routes"][0]["result_msg"])
print(doc[0]["routes"][0]["summary"])

morning_time = morning_time - int(doc[0]["routes"][0]["summary"]["duration"]) - 3600
total_time = int(doc[0]["routes"][0]["summary"]["duration"]) + 3600

<class 'float'>
0 길찾기 성공
{'origin': {'name': '', 'x': 126.49594267906363, 'y': 33.50593465303201}, 'destination': {'name': '', 'x': 126.48821557505798, 'y': 33.470643362077055}, 'waypoints': [], 'priority': 'RECOMMEND', 'bound': {'min_x': 126.47972101703017, 'min_y': 33.4691871043314, 'max_x': 126.49678973945115, 'max_y': 33.50673831393813}, 'fare': {'taxi': 7500, 'toll': 0}, 'distance': 6510, 'duration': 1091}


In [131]:
#오전 일정
# 오전 부분 = 오전 시간을 모두 채울 떄까지 진행
for l in range(10):
    min = 10000
    for k in range(len(sort2_place)):
        dist = cal_dist(last_end_place["latitude"], last_end_place["longitude"], sort2_place[k]["latitude"],sort2_place[k]["longitude"])
        if dist >= 1 and dist <= 10:
            if dist <= min:
                min = dist
                end_place = sort2_place[k]
                num = k
    
    #마지막으로 이동했던 장소와 다음 이동할 장소의 이동정보 url
              
    url = "https://apis-navi.kakaomobility.com/v1/directions?origin={0},{1}&destination={2},{3}".format(last_end_place["longitude"],last_end_place["latitude"],sort2_place[num]["longitude"], sort2_place[num]["latitude"]) + "&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"            
    # GET을 이용하여 정보 불러오기
    res = requests.get(url, headers=headers)
    if morning_time - int(json.loads(res.text)["routes"][0]["summary"]["duration"]) - 3600 + 600 < 0:
        break
    else:
        morning_time = morning_time - int(json.loads(res.text)["routes"][0]["summary"]["duration"]) - 3600
        total_time += int(doc[0]["routes"][0]["summary"]["duration"]) + 3600
        
    last_end_place = end_place
    
    print(sort2_place[num]["title"])
    del sort2_place[num]
    
    # Json 형식으로 불러오기
    #불러온 이동정보를 doc에 추가
    doc.append(json.loads(res.text))
    print(json.loads(res.text)["routes"][0]["result_code"], json.loads(res.text)["routes"][0]["result_msg"])
print(total_time)

관덕정
0 길찾기 성공
렛츠런파크제주
0 길찾기 성공
14073


In [132]:
#점심 식사
#식당을 저장할 리스트
restaurant = []

with open('data_noncafe.json', 'r', encoding='utf-8') as f:
    restaurant_data = json.load(f)
    
    for i in range(len(restaurant_data["items"])):
        restaurant.append(restaurant_data["items"][i])
    
    #평점 높은순 정렬
    
sort_restaurant = sorted(restaurant, key=itemgetter("kakaorate"), reverse=True)


min = 10000

for n in range(len(sort_restaurant)):
    dist = cal_dist(last_end_place["latitude"], last_end_place["longitude"], sort_restaurant[n]["latitude"], sort_restaurant[n]["longitude"]) 
    if dist >= 1 and dist <= 10:
        if dist <= min:
            min = dist
            end_place = sort_restaurant[n]
            num = n
        


url = "https://apis-navi.kakaomobility.com/v1/directions?origin={0},{1}&destination={2},{3}".format(last_end_place["longitude"],last_end_place["latitude"],sort_restaurant[num]["longitude"], sort_restaurant[num]["latitude"]) + "&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"            
# GET을 이용하여 정보 불러오기
res = requests.get(url, headers=headers)
print(sort_restaurant[num]["title"])

del sort_restaurant[num]
last_end_place = end_place
doc.append(json.loads(res.text))
print(json.loads(res.text)["routes"][0]["result_code"], json.loads(res.text)["routes"][0]["result_msg"])
total_time += int(doc[0]["routes"][0]["summary"]["duration"]) + 3600

모리노아루요
0 길찾기 성공


In [138]:
#오후 일정
afternoon_time = 18 - start_time - (total_time/3600)
afternoon_time = afternoon_time * 3600

for l in range(10):
    min = 10000
    for k in range(len(sort2_place)):
        dist = cal_dist(last_end_place["latitude"], last_end_place["longitude"], sort2_place[k]["latitude"],sort2_place[k]["longitude"])
        if dist >= 1 and dist <= 10:
            if dist <= min:
                min = dist
                end_place = sort2_place[k]
                num = k
    
    #마지막으로 이동했던 장소와 다음 이동할 장소의 이동정보 url
              
    url = "https://apis-navi.kakaomobility.com/v1/directions?origin={0},{1}&destination={2},{3}".format(last_end_place["longitude"],last_end_place["latitude"],sort2_place[num]["longitude"], sort2_place[num]["latitude"]) + "&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"            
    # GET을 이용하여 정보 불러오기
    res = requests.get(url, headers=headers)
    if afternoon_time - int(json.loads(res.text)["routes"][0]["summary"]["duration"]) - 3600 + 600 < 0:
        break
    else:
        afternoon_time = afternoon_time - int(json.loads(res.text)["routes"][0]["summary"]["duration"]) - 3600
        total_time += int(doc[0]["routes"][0]["summary"]["duration"]) + 3600
        
    last_end_place = end_place
    
    print(sort2_place[num]["title"])
    del sort2_place[num]
    
    # Json 형식으로 불러오기
    #불러온 이동정보를 doc에 추가
    doc.append(json.loads(res.text))
    print(json.loads(res.text)["routes"][0]["result_code"], json.loads(res.text)["routes"][0]["result_msg"])

도치돌알파카목장
0 길찾기 성공
9.81파크
0 길찾기 성공
곽지해수욕장
0 길찾기 성공
제주카약올레
0 길찾기 성공


In [141]:
#저녁 식사
#식당을 저장할 리스트

min = 10000
for n in range(len(sort_restaurant)):
    dist = cal_dist(last_end_place["latitude"], last_end_place["longitude"], sort_restaurant[n]["latitude"], sort_restaurant[n]["longitude"]) 
    if dist >= 1 and dist <= 10:
        if dist <= min:
            min = dist
            end_place = sort_restaurant[n]
            num = n
        


url = "https://apis-navi.kakaomobility.com/v1/directions?origin={0},{1}&destination={2},{3}".format(last_end_place["longitude"],last_end_place["latitude"],sort_restaurant[num]["longitude"], sort_restaurant[num]["latitude"]) + "&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"            
# GET을 이용하여 정보 불러오기
res = requests.get(url, headers=headers)
print(sort_restaurant[num]["title"])

del sort_restaurant[num]
last_end_place = end_place
doc.append(json.loads(res.text))
print(json.loads(res.text)["routes"][0]["result_code"], json.loads(res.text)["routes"][0]["result_msg"])
total_time += int(doc[0]["routes"][0]["summary"]["duration"]) + 3600

카페태희
0 길찾기 성공


In [145]:
#숙소 
accommodation = []

with open('data_c3.json', 'r', encoding='utf-8') as f:
    accommodation_data = json.load(f)
    
    for i in range(len(accommodation_data["items"])):
        accommodation.append(accommodation_data["items"][i])
    
    #평점 높은순 정렬
    
sort_accommodation = sorted(accommodation, key=itemgetter("kakaorate"), reverse=True)


min = 10000

for n in range(len(sort_accommodation)):
    dist = cal_dist(last_end_place["latitude"], last_end_place["longitude"], sort_accommodation[n]["latitude"], sort_accommodation[n]["longitude"]) 
    if dist >= 1 and dist <= 10:
        if dist <= min:
            min = dist
            end_place = sort_accommodation[n]
            num = n
        


url = "https://apis-navi.kakaomobility.com/v1/directions?origin={0},{1}&destination={2},{3}".format(last_end_place["longitude"],last_end_place["latitude"],sort_accommodation[num]["longitude"], sort_accommodation[num]["latitude"]) + "&waypoints=&priority=RECOMMEND&car_fuel=GASOLINE&car_hipass=false&alternatives=false&road_details=false"            
# GET을 이용하여 정보 불러오기
res = requests.get(url, headers=headers)
print(sort_restaurant[num]["title"])

del sort_restaurant[num]
last_end_place = end_place
doc.append(json.loads(res.text))
print(json.loads(res.text)["routes"][0]["result_code"], json.loads(res.text)["routes"][0]["result_msg"])
total_time += int(doc[0]["routes"][0]["summary"]["duration"]) + 3600

신우성흑돼지
0 길찾기 성공


In [146]:
#folium map open
map = folium.Map(location=[start_latitude, start_longitude], zoom_start=11)

#read doc
for n in range(len(doc)):
    
    #doc에 저장된 이동정보를 불러와서 마커 추가
    origin = doc[n]["routes"][0]["summary"]["origin"]
    destination = doc[n]["routes"][0]["summary"]["destination"]

    folium.Marker(location=[origin['y'], origin['x']], popup=n).add_to(map)
    folium.Marker(location=[destination['y'], destination['x']]).add_to(map)
    
#folium.PolyLine(vertexes, color='red').add_to(map)
map

In [140]:
#총 이동시간 계산
#duration: 이동시간(초)
print("총 소요시간: {}".format(int(total_time/60)) + "분")

총 소요시간: 625분


In [43]:
#경로 추가 버전

col = ["red","blue","green","purple","orange","white","pink"]
a = None
a = random.sample(range(0,7),6)
#folium map open
map = folium.Map(location=[start_latitude, start_longitude], zoom_start=11)

#read doc
for n in range(len(doc)):
    
    #doc에 저장된 이동정보를 불러와서 마커 추가
    origin = doc[n]["routes"][0]["summary"]["origin"]
    destination = doc[n]["routes"][0]["summary"]["destination"]

    folium.Marker(location=[origin['y'], origin['x']]).add_to(map)
    folium.Marker(location=[destination['y'], destination['x']]).add_to(map)
    
    
    
    vertexes = []
    #각 이동정보에서 경로를 얻어와 저장
    for road in doc[n]['routes'][0]['sections'][0]['roads']:
        r = road['vertexes']
        for i in range(0, len(r), 2):
            vertexes.append((r[i+1], r[i]))
            
    #경로를 그려줌        
    folium.PolyLine(vertexes, color=col[1]).add_to(map)
map